# Storytelling from Geojson and Choropleth Maps

In [ ]:
!unzip datasets.zip

## Introduction




In this mission, we'll analyze geographic data on brazilian municipalities. The dataset comes from [IBGE - Instituto Brasileiro de Geografia e Estatística](https://downloads.ibge.gov.br/downloads_estatisticas.htm). The goal is to expand our skills about visualization on [choropleth maps](https://en.wikipedia.org/wiki/Choropleth_map). 

A choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income. 



In [ ]:
!pip install shapely

In [ ]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

In [ ]:
# dataset name
dataset_pop_2017 = os.path.join('data', 'population_2017.csv')

# read the data to a dataframe
data2017 = pd.read_csv(dataset_pop_2017)

# eliminate spaces in name of columns
data2017.columns = [cols.replace(' ', '_') for cols in data2017.columns]

data2017.head()


In [ ]:
# filtering data about RN state
dataRN = data2017[data2017['UF'] == 'RN']

# sort dataset by city name
dataRN = dataRN.sort_values('NOME_DO_MUNICÍPIO')

dataRN.head()



## Geodata BR - Brazil




This project contains files [Geojson](http://geojson.org/) with the perimeters
of Brazilian municipalities group by state.

source: https://github.com/tbrugz/geodata-br



### North Region
* AC / Acre - [geojson/geojs-12-mun.json](geojson/geojs-12-mun.json)
* AM / Amazonas - [geojson/geojs-13-mun.json](geojson/geojs-13-mun.json)
* AP / Amapá - [geojson/geojs-16-mun.json](geojson/geojs-16-mun.json)
* PA / Pará  - [geojson/geojs-15-mun.json](geojson/geojs-15-mun.json)
* RO / Rondônia - [geojson/geojs-11-mun.json](geojson/geojs-11-mun.json)
* RR / Roraima - [geojson/geojs-14-mun.json](geojson/geojs-14-mun.json)
* TO / Tocantins - [geojson/geojs-17-mun.json](geojson/geojs-17-mun.json)


### Northeast Region
* AL / Alagoas - [geojson/geojs-27-mun.json](geojson/geojs-27-mun.json)
* BA / Bahia - [geojson/geojs-29-mun.json](geojson/geojs-29-mun.json)
* CE / Ceará - [geojson/geojs-23-mun.json](geojson/geojs-23-mun.json)
* MA / Maranhão - [geojson/geojs-21-mun.json](geojson/geojs-21-mun.json)
* PB / Paraíba - [geojson/geojs-25-mun.json](geojson/geojs-25-mun.json)
* PE / Pernambuco - [geojson/geojs-26-mun.json](geojson/geojs-26-mun.json)
* PI / Piauí - [geojson/geojs-22-mun.json](geojson/geojs-22-mun.json)
* RN / Rio Grande do Norte - [geojson/geojs-24-mun.json](geojson/geojs-24-mun.json)
* SE / Sergipe - [geojson/geojs-28-mun.json](geojson/geojs-28-mun.json)


### Southeast Region
* ES / Espíriro Santo - [geojson/geojs-32-mun.json](geojson/geojs-32-mun.json)
* MG / Minas Gerais - [geojson/geojs-31-mun.json](geojson/geojs-31-mun.json)
* RJ / Rio de Janeiro - [geojson/geojs-33-mun.json](geojson/geojs-33-mun.json)
* SP / São Paulo - [geojson/geojs-35-mun.json](geojson/geojs-35-mun.json)


### South Region
* PR / Paraná - [geojson/geojs-41-mun.json](geojson/geojs-41-mun.json)
* RS / Rio Grande do Sul - [geojson/geojs-43-mun.json](geojson/geojs-43-mun.json)
* SC / Santa Catarina - [geojson/geojs-42-mun.json](geojson/geojs-42-mun.json)


### Central-west Region
* DF / Distrito Federal - [geojson/geojs-53-mun.json](geojson/geojs-53-mun.json) 
* GO / Goiás - [geojson/geojs-52-mun.json](geojson/geojs-52-mun.json)
* MT / Mato Grosso - [geojson/geojs-51-mun.json](geojson/geojs-51-mun.json)
* MS / Mato Grosso do Sul - [geojson/geojs-50-mun.json](geojson/geojs-50-mun.json)


### Brazil
* BR / Brazil - [geojson/geojs-100-mun.json](geojson/geojs-100-mun.json)




## Customization 



If you need, it is possible generate and customize GeoJson files from online editors. The project [GeoJson.io](http://geojson.io/) is an excelent path to give the first steps. 



## Importing GeoJson files


In [ ]:
# searching the files in geojson/geojs-xx-mun.json
br_states = os.path.join('geojson', 'geojs-24-mun.json')

# load the data and use 'latin-1'encoding because the accent
geo_json_data = json.load(open(br_states,encoding='latin-1'))


In [ ]:
geo_json_data

In [ ]:
# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_data['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_data['features'][112]['properties']['name'] = 'Serra Caiada'


In [ ]:
cities = []
# list all cities in the state
for city in geo_json_data['features']:
        cities.append(city['properties']['description'])
cities


In [ ]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_data).add_to(m)

m

### Importing Geojson files from overpass-turbo project



Source: http://overpass-turbo.eu/

Query to [Natal neighborhoods](http://wiki.openstreetmap.org/wiki/Natal#Bairros):
```python
[out:json][timeout:25];
{{geocodeArea:Natal RN Brasil}}->.searchArea;
(
  relation["admin_level"="10"](area.searchArea);
);
out body;
>;
out skel qt;
```

In [ ]:
# import geojson file about natal neighborhood
natal_neigh = os.path.join('geojson', 'natal.geojson')

# load the data and use 'UTF-8'encoding
geo_json_natal = json.load(open(natal_neigh,encoding='UTF-8'))


In [ ]:
neighborhood = []
# list all neighborhoods
for neigh in geo_json_natal['features']:
    neighborhood.append(neigh['properties']['name'])
neighborhood


In [ ]:
len(neighborhood)

In [ ]:
# return a number of points inside the polygon
def generate_random(number, polygon, neighborhood):
    list_of_points = []
    minx, miny, maxx, maxy = polygon.bounds
    counter = 0
    while counter < number:
        x = random.uniform(minx, maxx)
        y = random.uniform(miny, maxy)
        pnt = Point(x, y)
        if polygon.contains(pnt):
            list_of_points.append([x,y,neighborhood])
            counter += 1
    return list_of_points

In [ ]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=11,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_natal).add_to(m)
m

In [ ]:
number_of_points = 3

# search all features
for feature in geo_json_natal['features']:
    # get the name of neighborhood
    neighborhood = feature['properties']['name']
    # take the coordinates (lat,log) of neighborhood
    geom = feature['geometry']['coordinates']
    # create a polygon using all coordinates
    polygon = Polygon(geom[0])
    # return number_of_points by neighborhood as a list [[log,lat],....]
    points = generate_random(number_of_points,polygon, neighborhood)
    # iterate over all points and print in the map
    for i,value in enumerate(points):
        log, lat, name = value 
        # Draw a small circle
        folium.CircleMarker([lat,log],
                    radius=2,
                    popup='%s %s%d' % (name, '#', i),
                   color='red').add_to(m)
m

## Drawing the Choropleth

In [ ]:
# verify again the datarn
dataRN.head()

Now we need to create a function that maps one value to a RGB color (of the form #RRGGBB). For this, we'll use colormap tools from branca.colormap.

In [ ]:
# colormap yellow and green (YlGn)
colormap = linear.YlGn_03.scale(
    dataRN.POPULAÇÃO_ESTIMADA.min(),
    dataRN.POPULAÇÃO_ESTIMADA.max())

print(colormap(5000.0))

colormap


We need also to convert the table into a dictionary in order to map a feature to it's population estimation.

In [ ]:
population_dict = dataRN.set_index('NOME_DO_MUNICÍPIO')['POPULAÇÃO_ESTIMADA']
population_dict[:3]

Now we can do the choropleth.

In [ ]:
# option 1

# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(
    geo_json_data,
    name='Population estimation of RN State in 2017',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['description']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9
    },
    tooltip=folium.GeoJsonTooltip(fields=['name'],
                                              labels=False,
                                              sticky=False)
).add_to(m)

colormap.caption = 'Population estimation (2017)'
colormap.add_to(m)

folium.LayerControl().add_to(m)

m



Then, in playing with keyword arguments, you can get a choropleth in (almost) one line :


In [ ]:
# option 2

# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(dataRN['POPULAÇÃO_ESTIMADA'].min(),
                              dataRN['POPULAÇÃO_ESTIMADA'].max(), 6, dtype=int).tolist()


m.choropleth(
    geo_data=geo_json_data,
    data=dataRN,
    columns=['NOME_DO_MUNICÍPIO', 'POPULAÇÃO_ESTIMADA'],
    key_on='feature.properties.description',
    fill_color='YlGn',
    legend_name='Population estimation (2017)',
    highlight=True,
    threshold_scale = threshold_scale
)

m



**Exercise**

<left><img width="100" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>



1. Extrapolate the choropleth map (population estimation) in section 5 to Northeast region. 
2. Choice other metrics from http://dados.gov.br/

In [ ]:
# put your code here